# Names and coordinates of Swiss cities

In this notebook, we will import the name and the GPS coordinates from all municipalities in Switzerland.

We will import a CSV file and convert it directly into a dataframe.
Our source is the Swiss Federal Office of Topography. The file is updated every month.

## Import the data

In [ ]:
#Install the required libraries.
import sqlalchemy
import pandas as pd
import io
import requests
import zipfile
import mysql.connector
from bs4 import BeautifulSoup
from sqlalchemy.sql import text

In [ ]:
#Get the connection to the website
url = 'https://www.cadastre.ch/de/services/service/registry/plz.html'

#Accessing the entire website
website = requests.get(url)

#Creating a beautiful soup object with the webpage, using the html parser
soup = BeautifulSoup(website.content, 'html.parser')

#Finding the right section
section = soup.find('div', class_= 'parsys_column row')

#Finding the section with the link to the file
link = section.find('a', string = 'CSV (Excel) WGS84 ')

#Extract the link with the desired data
data_file = link['href']

#Unzip the zip file and store it in the same directory as this file is stored
get_data = requests.get(data_file)
content = zipfile.ZipFile(io.BytesIO(get_data.content))
data_folder = content.extractall()

#Load Data into Pandas DataFrame
raw_data = pd.read_csv('PLZO_CSV_WGS84/PLZO_CSV_WGS84.csv', sep=';', engine='python')
raw_data.head(3)

In [ ]:
#Just take those columns that are needed
raw_data = raw_data[['Ortschaftsname', 'PLZ', 'E', 'N']]

#Rename E and N to Longtidue and Latitude for a better understanding of the data
koord_data = raw_data.rename(columns={
    'E' : 'Longitude', 
    'N' : 'Latitude'}
    )

koord_data.head(3)

In [ ]:
#Replace all ä,ö,ü with ae, oe, ue
koord_data = koord_data.replace('ä', 'ae', regex=True)
koord_data = koord_data.replace('ö', 'oe', regex=True)
koord_data = koord_data.replace('ü', 'ue', regex=True)

## Data in the table
- 1st column is the name of the village
- 2nd column are the ZIP codes of the villages
- 3rd and 4th column contain the GPS coordinates: Latitude and Longitude


In [ ]:
#Create URI for DataBase connection
database_username ='climate_change'
database_password = 'FHNW_climate_20'
database_ip = '45.32.156.57'
database_port = '3306'
database_name = 'Climate_Change'

database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [ ]:
#Evoke connection to DataBase
con = database_connection.connect()

# Create Table in MySQL DataBase
In order to safe the data into a DB it is necessary to define the field correctly, otherwise the risk can be that wrong types of field might consume too much ressources and slow down your system. 


In [ ]:
#Query to create a table in the Climate Change Schema, with correct types
query = text("""
CREATE TABLE `Climate_Change`.`coordinates` (
  `Ortschaftsname` VARCHAR(40) NOT NULL,
  `PLZ` INT NULL,
  `Longitude` FLOAT NULL,
  `Latitude` FLOAT NULL,
  PRIMARY KEY (`Ortschaftsname`))
ENGINE = InnoDB
DEFAULT CHARACTER SET = utf8;
""")

In [ ]:
#Execute Query to create Table in DataBase
con.execute(query)

In [ ]:
#Send DataFrame to DataBase by using Pandas functionality
koord_data.to_sql(con=database_connection, name='coordinates', if_exists='replace')

## SQL Queries for different needs
In this section we will safe the most used Queries in order to have a uniform way of operating with them

In [ ]:
#Deleting all existing row in the coordinates table
delet_rows = text("""
DELETE FROM Climate_Change.coordinates;
""")

#Selecting all village names from coordinates table, which will be used for the drop down menu
village_names = 'SELECT Ortschaftsname FROM Climate_Change.coordinates'


In [ ]:
#Exectuing Village Name extraction
select_coordinates = pd.read_sql(village_names, con=con)
select_coordinates.head(3)